## Operaciones con DataFrames
Veremos distintas operaciones que se pueden hacer con los DataFrames:

  - Filtrado de filas
  - Ordenación y agrupamiento
  - Joins
  - Funciones escalares y agregados
  - Manejo de tipos complejos
  - Funciones de ventana
  - Funciones definidas por el usuario
 
Acabaremos viendo como usar consultas SQL sobre DataFrames
  

In [21]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os

# Elegir el máster de Spark dependiendo de si se ha definido la variable de entorno HADOOP_CONF_DIR o YARN_CONF_DIR
SPARK_MASTER: str = 'local[*]'
if 'HADOOP_CONF_DIR' in os.environ or 'YARN_CONF_DIR' in os.environ:
  SPARK_MASTER = 'yarn'

# Creamos un objeto SparkSession (o lo obtenemos si ya está creado)
spark: SparkSession = SparkSession \
  .builder \
  .appName("Mi aplicacion") \
  .config("spark.rdd.compress", "true") \
  .config("spark.executor.memory", "3g") \
  .config("spark.driver.memory", "3g") \
  .master(SPARK_MASTER) \
  .getOrCreate()

sc: SparkContext = spark.sparkContext

In [22]:
%%sh
rm -rf /tmp/tcdm-public
git clone -b 24-25 --single-branch --depth 1 https://github.com/dsevilla/tcdm-public.git /tmp/tcdm-public

Clonando en '/tmp/tcdm-public'...


In [23]:
# Recupero el DataFrame leyéndolo del formato parquet
from pyspark.sql.dataframe import DataFrame

dfSE: DataFrame = spark.read\
            .format("parquet")\
            .option("mode", "FAILFAST")\
            .load("/tmp/tcdm-public/datos/dfSE.parquet")
dfSE.cache()

24/11/12 17:00:20 WARN CacheManager: Asked to cache already cached data.


DataFrame[Id: int, AcceptedAnswerId: int, AnswerCount: int, Body: string, ClosedDate: timestamp, CommentCount: int, CommunityOwnedDate: timestamp, ContentLicense: string, Fecha_de_creación: timestamp, FavoriteCount: int, LastActivityDate: timestamp, LastEditDate: timestamp, LastEditorDisplayName: string, LastEditorUserId: int, OwnerDisplayName: string, OwnerUserId: int, ParentId: int, PostTypeId: int, Score: int, Tags: string, Title: string, ViewCount: int]

In [24]:
dfSE.show(5)
dfSE.printSchema()

+------+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+--------------------+---------+
|    Id|AcceptedAnswerId|AnswerCount|                Body|ClosedDate|CommentCount|CommunityOwnedDate|ContentLicense|   Fecha_de_creación|FavoriteCount|    LastActivityDate|        LastEditDate|LastEditorDisplayName|LastEditorUserId|OwnerDisplayName|OwnerUserId|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+------+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+------------

In [25]:
dfSE.count()

410346

## Operaciones de filtrado

In [26]:
# Selecciona los post que tengan la palabra Italiano en su cuerpo
from pyspark.sql.column import Column
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

colCuerpo: Column = col("Body")
dfConItaliano: DataFrame = dfSE.filter(colCuerpo.like('%Italiano%'))

print("Número de posts con la palabra Italiano: {0}\n"\
      .format(dfConItaliano.count()))

print("Una de las filas")
dfConItaliano.take(1)

Número de posts con la palabra Italiano: 32

Una de las filas


[Row(Id=396443, AcceptedAnswerId=396463, AnswerCount=1, Body='<p>Tengo un archivo que he creado, con el nombre <code>available_languages.php</code>. Este archivo está en el directorio <code>/config</code>, y su contenido es el siguiente:</p><br/><pre><code><?php<br/>    return [<br/>        \'idiomas_disponibles\' => [<br/>            \'ca\' => [<br/>                \'code\' => \'ca\',<br/>                \'variable_name\' => \'Catalán\',<br/>                \'image_name\' => \'ca.jpg\',<br/>            ],<br/>            \'da\' => [<br/>                \'code\' => \'da\',<br/>                \'variable_name\' => \'Danés\',<br/>                \'image_name\' => \'da.jpg\',<br/>            ],<br/>            \'de\' => [<br/>                \'code\' => \'de\',<br/>                \'variable_name\' => \'Alemán\',<br/>                \'image_name\' => \'de.jpg\',<br/>            ],<br/>            \'en\' => [<br/>                \'code\' => \'en\',<br/>                \'variable_name\' => 

In [27]:
# Obtenemos las preguntas (PostTypeId == 1) que tienen una respuesta aceptada (AcceptedAnswerId != null)
# Nota: where() es un alias de filter()

postTypeIdCol: Column = col("PostTypeId")
acceptedAnswerIdCol: Column = col("AcceptedAnswerId")

questionsWithAcceptedAnswersDf: DataFrame = (dfSE
                    .where((postTypeIdCol == 1) & (acceptedAnswerIdCol.isNotNull()))
                    .withColumnRenamed("CreationDate", "Fecha_de_creación"))

print("Número de preguntas con respuesta aceptada: {0}"\
      .format(questionsWithAcceptedAnswersDf.count()))

questionsWithAcceptedAnswersDf.cache()

(questionsWithAcceptedAnswersDf
        .select("Fecha_de_creación", postTypeIdCol.alias("Tipo Post"), acceptedAnswerIdCol)
        .show(truncate=False))

Número de preguntas con respuesta aceptada: 82491


24/11/12 17:00:22 WARN CacheManager: Asked to cache already cached data.


+-----------------------+---------+----------------+
|Fecha_de_creación      |Tipo Post|AcceptedAnswerId|
+-----------------------+---------+----------------+
|2020-08-03 19:45:29.76 |1        |379397          |
|2020-08-03 20:38:32.387|1        |379500          |
|2020-08-03 22:45:38.533|1        |379443          |
|2020-08-03 22:52:19.433|1        |379436          |
|2020-08-03 22:56:06.02 |1        |379442          |
|2020-08-03 23:44:40.883|1        |379444          |
|2020-08-04 01:11:28.32 |1        |379457          |
|2020-08-04 02:55:40.433|1        |379477          |
|2020-08-04 04:20:51.807|1        |379489          |
|2020-08-04 04:41:24.703|1        |379486          |
|2020-08-04 05:37:08.907|1        |379791          |
|2020-08-04 06:54:14.673|1        |379507          |
|2020-08-04 07:06:42.663|1        |379554          |
|2020-08-04 07:19:01.563|1        |379538          |
|2020-08-04 09:11:52.593|1        |379516          |
|2020-08-04 09:33:21.117|1        |379624     

In [28]:
# Nos quedamos con las entradas correspondientes a junio de 2016
from datetime import date

fechaCreacionCol = col("Fecha_de_creación")

dfPregConRespAceptJun16: DataFrame = questionsWithAcceptedAnswersDf\
                    .filter((fechaCreacionCol >= date(2016,6,1)) &
                            (fechaCreacionCol <= date(2016,6,30)))

dfPregConRespAceptJun16.select(fechaCreacionCol, postTypeIdCol, acceptedAnswerIdCol)\
        .show(truncate=False)

+-----------------------+----------+----------------+
|Fecha_de_creación      |PostTypeId|AcceptedAnswerId|
+-----------------------+----------+----------------+
|2016-06-01 07:23:19.88 |1         |11532           |
|2016-06-01 08:00:58.677|1         |11538           |
|2016-06-01 08:43:45.403|1         |11544           |
|2016-06-01 09:00:34.167|1         |11546           |
|2016-06-01 13:26:14.547|1         |11566           |
|2016-06-01 14:03:25.49 |1         |11570           |
|2016-06-01 14:29:37.477|1         |11577           |
|2016-06-01 17:00:45.003|1         |11601           |
|2016-06-01 17:22:16.093|1         |11597           |
|2016-06-01 18:05:49.507|1         |11615           |
|2016-06-01 18:21:23.367|1         |11611           |
|2016-06-01 18:27:23.233|1         |11613           |
|2016-06-01 19:33:21.04 |1         |11663           |
|2016-06-01 20:00:19.05 |1         |11639           |
|2016-06-01 20:02:27.663|1         |11691           |
|2016-06-01 20:25:06.643|1  

In [29]:
# Añadimos una columna que contenga el ratio entre el número de vistas y el score
colNumVistas = col("ViewCount")
colPuntos = col("Score")
dfPregConRespAceptyRatio = questionsWithAcceptedAnswersDf.withColumn("ratio", colNumVistas/colPuntos)

# Muestra algunas columnas con ratio > 35
colRatio = col("ratio")
(dfPregConRespAceptyRatio.filter(colRatio > 35)
                        .select(fechaCreacionCol, colNumVistas, colPuntos, colRatio)
                        .show(truncate=False))

+-----------------------+---------+-----+------------------+
|Fecha_de_creación      |ViewCount|Score|ratio             |
+-----------------------+---------+-----+------------------+
|2020-08-03 22:52:19.433|225      |1    |225.0             |
|2020-08-04 01:11:28.32 |50       |1    |50.0              |
|2020-08-04 07:06:42.663|109      |1    |109.0             |
|2020-08-04 11:46:59.383|4651     |1    |4651.0            |
|2020-08-04 12:05:20.233|85       |1    |85.0              |
|2020-08-04 17:31:29.23 |348      |1    |348.0             |
|2020-08-04 18:42:51.897|383      |2    |191.5             |
|2020-08-04 18:56:22.563|358      |1    |358.0             |
|2020-08-04 20:04:33.76 |232      |1    |232.0             |
|2020-08-04 20:39:55.82 |146      |1    |146.0             |
|2020-08-05 00:45:17.277|81       |1    |81.0              |
|2020-08-05 04:28:38.203|1373     |1    |1373.0            |
|2020-08-05 11:34:05.64 |60       |1    |60.0              |
|2020-08-05 14:43:05.277

## Operaciones de ordenación y agrupamiento

In [30]:
# Ordenamos por viewCount
questionsWithAcceptedAnswersDf.orderBy(colNumVistas.desc())\
                  .select(fechaCreacionCol, colNumVistas)\
                  .show(10, truncate=False)

+-----------------------+---------+
|Fecha_de_creación      |ViewCount|
+-----------------------+---------+
|2017-01-25 01:00:14.09 |496200   |
|2015-12-21 22:11:08.127|400626   |
|2016-05-23 18:34:19.627|334608   |
|2017-02-10 14:57:10.55 |295361   |
|2016-10-23 16:20:32.963|280218   |
|2015-12-02 12:35:37.43 |256224   |
|2016-07-25 07:58:15.75 |252099   |
|2018-03-04 23:57:12.52 |247708   |
|2016-12-29 11:04:12.557|238862   |
|2016-11-11 22:42:23.697|232373   |
+-----------------------+---------+
only showing top 10 rows



In [31]:
# Creamos una agrupación por la columna OwnerUserId
from pyspark.sql.column import Column
from pyspark.sql.group import GroupedData

colUserId: Column = col("OwnerUserId")
grupoPorUsuario: GroupedData = questionsWithAcceptedAnswersDf.groupBy(colUserId)
print(type(grupoPorUsuario))

<class 'pyspark.sql.group.GroupedData'>


In [32]:
print("DataFrame con el número de posts por usuario")
dfPostPorUsuario: DataFrame = grupoPorUsuario.count()
dfPostPorUsuario.printSchema()

colNPosts: Column = col("count")
dfPostPorUsuario.select(colUserId.alias("Número de usuario"),
                        colNPosts.alias("Número de posts"))\
                .orderBy(colNPosts, ascending=False).show(10)

DataFrame con el número de posts por usuario
root
 |-- OwnerUserId: integer (nullable = true)
 |-- count: long (nullable = false)

+-----------------+---------------+
|Número de usuario|Número de posts|
+-----------------+---------------+
|             NULL|           1546|
|             2633|            271|
|             2230|            251|
|            95858|            235|
|           131982|            204|
|            38304|            197|
|             4048|            184|
|            22584|            173|
|            29967|            163|
|             5997|            162|
+-----------------+---------------+
only showing top 10 rows



In [33]:
print("DataFrame con la media de vistas por usuario")
dfAvgPorUsuario: DataFrame = grupoPorUsuario.avg("ViewCount")\
                    .withColumnRenamed("avg(ViewCount)", "Media_vistas")
dfAvgPorUsuario.orderBy("Media_vistas", ascending=False).show(10)

DataFrame con la media de vistas por usuario
+-----------+-----------------+
|OwnerUserId|     Media_vistas|
+-----------+-----------------+
|      14311|         194210.0|
|      58631|         184583.0|
|      22731|         116308.5|
|      38468|         106566.0|
|      19586|         102870.0|
|      19912|94980.66666666667|
|      29298|          94209.0|
|       2027|          93271.5|
|       5350|          88564.0|
|      34152|          87270.0|
+-----------+-----------------+
only showing top 10 rows



In [34]:
# El método agg permite hacer varias operaciones de agrupamiento, expresadas como un diccionario {nombre_columna:operacion}
print("Obtenemos las tablas anteriores con una sola operación")
dfCountyAvg: DataFrame = grupoPorUsuario.agg({"OwnerUserId":"count", "ViewCount":"avg"})
dfCountyAvg.printSchema()

colCount: Column = col("count(OwnerUserId)")
colMedia: Column = col("avg(ViewCount)")
dfCountyAvg.select(colUserId.alias("Número de usuario"),
                   colCount.alias("Número de posts"),
                   colMedia.alias("Media de vistas"))\
                  .orderBy(colUserId).show()


Obtenemos las tablas anteriores con una sola operación
root
 |-- OwnerUserId: integer (nullable = true)
 |-- count(OwnerUserId): long (nullable = false)
 |-- avg(ViewCount): double (nullable = true)

+-----------------+---------------+------------------+
|Número de usuario|Número de posts|   Media de vistas|
+-----------------+---------------+------------------+
|             NULL|              0|1683.5103492884864|
|               20|              2|            1737.0|
|               21|             20|          12629.75|
|               22|              7| 877.5714285714286|
|               23|              7| 622.1428571428571|
|               24|             18|            8439.0|
|               25|             10|            7243.5|
|               27|              1|             335.0|
|               28|              1|             421.0|
|               29|              3|             801.0|
|               34|              1|            1926.0|
|               35|           

In [35]:
# Agrupación sobre dos columnas
dfSE.groupBy(colUserId, postTypeIdCol)\
    .count()\
    .orderBy(colUserId.asc(), postTypeIdCol.desc())\
    .show()

+-----------+----------+-----+
|OwnerUserId|PostTypeId|count|
+-----------+----------+-----+
|       NULL|         6|    1|
|       NULL|         5|   70|
|       NULL|         4|   69|
|       NULL|         2| 3394|
|       NULL|         1| 2933|
|         -1|         7|    4|
|         -1|         5|  437|
|         -1|         4|   99|
|         20|         6|    1|
|         20|         5|    3|
|         20|         4|    7|
|         20|         2|  248|
|         20|         1|    3|
|         21|         5|   17|
|         21|         4|   14|
|         21|         2| 1091|
|         21|         1|   20|
|         22|         5|    2|
|         22|         4|    2|
|         22|         2|    9|
+-----------+----------+-----+
only showing top 20 rows



Una descripción de las funciones que se pueden usar con GroupedData está en https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.GroupedData.html 

### Extensiones del groupBy

Funciones `rollup` y `cube`

#### Rollup

Incluye filas adicionales con agregados por la primera columna

In [36]:
# Contar para cada usuario el número de preguntas (PostTypeId = 1) y el número de respuestas (PostTypeId = 2)
from pyspark.sql.group import GroupedData

rollupPorUsuarioyTipoPost: GroupedData = dfSE.rollup("OwnerUserId", "PostTypeId")
print(type(rollupPorUsuarioyTipoPost))

<class 'pyspark.sql.group.GroupedData'>


In [37]:
# DataFrame con el número de post por usuario y tipo pregunta
# Los campos a null son de agregación, por ejemplo:
# null null = todos los posts
# 4    null = todos los posts del usuario con id 4
# 4    1    = todos los post de tipo 1 del usuario 4
# NOTA: aparecen posts con tipo 4 y 5, no se a que corresponden
dfPostPorUsuarioyTipo: DataFrame = rollupPorUsuarioyTipoPost.count()
dfPostPorUsuarioyTipo.printSchema()
dfPostPorUsuarioyTipo.select(colUserId.alias("Número de usuario"),
                             postTypeIdCol.alias("Tipo de post"),
                             colNPosts.alias("Número de posts"))\
                     .orderBy(colUserId,postTypeIdCol)\
                     .show(100)

root
 |-- OwnerUserId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- count: long (nullable = false)

+-----------------+------------+---------------+
|Número de usuario|Tipo de post|Número de posts|
+-----------------+------------+---------------+
|             NULL|        NULL|           6467|
|             NULL|        NULL|         410346|
|             NULL|           1|           2933|
|             NULL|           2|           3394|
|             NULL|           4|             69|
|             NULL|           5|             70|
|             NULL|           6|              1|
|               -1|        NULL|            540|
|               -1|           4|             99|
|               -1|           5|            437|
|               -1|           7|              4|
|               20|        NULL|            262|
|               20|           1|              3|
|               20|           2|            248|
|               20|           4|      

#### Cubes

Similar al Rollups, pero recorriendo todas las dimensiones

In [38]:
from pyspark.sql.group import GroupedData

grupoPorUsuarioyTipoPost: GroupedData = dfSE.cube("OwnerUserId", "PostTypeId")
print(type(grupoPorUsuarioyTipoPost))

<class 'pyspark.sql.group.GroupedData'>


In [39]:
# DataFrame con el número de post por usuario y tipo pregunta
# Los campos a null son de agregación, por ejemplo:
# null null = todas los posts
# null 1    = todos los post de tipo 1
# 4    null = todos los posts del usuario con id 4
# 4    1    = todos los post de tipo 1 del usuario 4
# NOTA: aparecen posts con tipo 4 y 5, no se a que corresponden
dfPostPorUsuarioyTipo: DataFrame = grupoPorUsuarioyTipoPost.count()
dfPostPorUsuarioyTipo.printSchema()
dfPostPorUsuarioyTipo.select(colUserId.alias("Número de usuario"),
                             postTypeIdCol.alias("Tipo de post"),
                             colNPosts.alias("Número de posts"))\
                     .orderBy(colUserId,postTypeIdCol)\
                     .show(100)

root
 |-- OwnerUserId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- count: long (nullable = false)

+-----------------+------------+---------------+
|Número de usuario|Tipo de post|Número de posts|
+-----------------+------------+---------------+
|             NULL|        NULL|           6467|
|             NULL|        NULL|         410346|
|             NULL|           1|           2933|
|             NULL|           1|         194788|
|             NULL|           2|         211615|
|             NULL|           2|           3394|
|             NULL|           4|           1958|
|             NULL|           4|             69|
|             NULL|           5|           1958|
|             NULL|           5|             70|
|             NULL|           6|             23|
|             NULL|           6|              1|
|             NULL|           7|              4|
|               -1|        NULL|            540|
|               -1|           4|      

## Joins
Spark ofrece la posibilidad de realizar múltiples tipos de joins

  - inner, outer, left outer, right outer, left semi, left anti, cross

In [40]:
# Buscamos unir las preguntas con respuesta aceptada con la respuesta que se ha elegido como aceptada
# Unimos el campo AcceptedAnswerId de las preguntas con el campo id de las respuestas
dfPreguntas: DataFrame = questionsWithAcceptedAnswersDf\
                .select(colUserId, colCuerpo, acceptedAnswerIdCol)\
                .withColumnRenamed("OwnerUserId", "Usuario pregunta")\
                .withColumnRenamed("Body", "Pregunta")\
                .withColumnRenamed("AcceptedAnswerId", "ID Resp Aceptada")

colId: Column = col("id")
dfRespuestas: DataFrame = dfSE\
                .select(colId, colUserId, colCuerpo)\
                .where(postTypeIdCol == 2)\
                .withColumnRenamed("Id", "ID Respuesta")\
                .withColumnRenamed("OwnerUserId", "Usuario respuesta")\
                .withColumnRenamed("Body", "Respuesta")

nPreguntas = dfPreguntas.count()
AnswerCount = dfRespuestas.count()
print("Número de preguntas con respuesta aceptada = {0}".format(nPreguntas))
print("Número de respuestas = {0}".format(AnswerCount))

Número de preguntas con respuesta aceptada = 82491
Número de respuestas = 211615


In [41]:
# Expresión para el join
joinExpression: Column = dfPreguntas["ID Resp Aceptada"] == dfRespuestas["ID Respuesta"]

In [42]:
# Inner join
# Solo se incluyen las filas para las que la joinExpression es true
joinType = "inner"
dfInner: DataFrame = dfPreguntas.join(dfRespuestas, joinExpression, joinType)
nFilas = dfInner.count()
print("Número de filas = {0}".format(nFilas))
dfInner.show(100)

Número de filas = 82491


+----------------+--------------------+----------------+------------+-----------------+--------------------+
|Usuario pregunta|            Pregunta|ID Resp Aceptada|ID Respuesta|Usuario respuesta|           Respuesta|
+----------------+--------------------+----------------+------------+-----------------+--------------------+
|              29|<p>¿Cómo se hace ...|              31|          31|               37|<p>Podes cambiar ...|
|             305|<p>En <a href="/q...|             137|         137|              305|<p>De acuerdo con...|
|              83|<p>Supongamos que...|             192|         192|              320|<p>Fácil de escri...|
|             100|<p>Estoy observan...|             243|         243|               83|<p><sub>Respondo ...|
|             187|<p>Muchas veces u...|             412|         412|               24|<p>Segmentation f...|
|             342|<p>Cuando corro u...|             458|         458|              342|<p>Ya lo pude res...|
|             100|<

In [43]:
# Outer join
# Incluye todas las filas de ambos DataFrames.
# En el caso de que no haya equivalente el alguno de los DataFrame, se meten nulls
joinType = "outer"
dfOuter: DataFrame = dfPreguntas.join(dfRespuestas, joinExpression, joinType)
nFilas = dfOuter.count()
print("Número de filas = {0}".format(nFilas))
dfOuter.show(100)

Número de filas = 211615


+----------------+--------------------+----------------+------------+-----------------+--------------------+
|Usuario pregunta|            Pregunta|ID Resp Aceptada|ID Respuesta|Usuario respuesta|           Respuesta|
+----------------+--------------------+----------------+------------+-----------------+--------------------+
|              29|<p>¿Cómo se hace ...|              31|          31|               37|<p>Podes cambiar ...|
|            NULL|                NULL|            NULL|          44|               25|<p>Así es como lo...|
|            NULL|                NULL|            NULL|          76|               93|<p>Puedes usar <a...|
|            NULL|                NULL|            NULL|          78|               72|<p>Usa <a href="h...|
|            NULL|                NULL|            NULL|         108|              149|<p>Puedes aplicar...|
|            NULL|                NULL|            NULL|         126|              326|<p>No es necesari...|
|            NULL| 

In [44]:
# Left Outer join
# Incluye todas las filas del DataFrame de la izquierda (primer DataFrame)
# Si no hay equivalencia en el de la derecha, se pone null.
joinType = "left_outer"
dfLOuter = dfPreguntas.join(dfRespuestas, joinExpression, joinType)
nFilas = dfLOuter.count()
print("Número de filas = {0}".format(nFilas))
dfLOuter.show(100)

Número de filas = 82491


+----------------+--------------------+----------------+------------+-----------------+--------------------+
|Usuario pregunta|            Pregunta|ID Resp Aceptada|ID Respuesta|Usuario respuesta|           Respuesta|
+----------------+--------------------+----------------+------------+-----------------+--------------------+
|              29|<p>¿Cómo se hace ...|              31|          31|               37|<p>Podes cambiar ...|
|             305|<p>En <a href="/q...|             137|         137|              305|<p>De acuerdo con...|
|              83|<p>Supongamos que...|             192|         192|              320|<p>Fácil de escri...|
|             100|<p>Estoy observan...|             243|         243|               83|<p><sub>Respondo ...|
|             187|<p>Muchas veces u...|             412|         412|               24|<p>Segmentation f...|
|             342|<p>Cuando corro u...|             458|         458|              342|<p>Ya lo pude res...|
|             399|<

In [45]:
# Right Outer join
# Incluye todas las filas del DataFrame de la derecha (segundo DataFrame)
# Si no hay equivalencia en el de la izquierda, se pone null.
joinType = "right_outer"
dfROuter = dfPreguntas.join(dfRespuestas, joinExpression, joinType)
nFilas = dfROuter.count()
print("Número de filas = {0}".format(nFilas))
dfROuter.show(100)

Número de filas = 211615
+----------------+--------------------+----------------+------------+-----------------+--------------------+
|Usuario pregunta|            Pregunta|ID Resp Aceptada|ID Respuesta|Usuario respuesta|           Respuesta|
+----------------+--------------------+----------------+------------+-----------------+--------------------+
|              29|<p>¿Cómo se hace ...|              31|          31|               37|<p>Podes cambiar ...|
|            NULL|                NULL|            NULL|          44|               25|<p>Así es como lo...|
|            NULL|                NULL|            NULL|          76|               93|<p>Puedes usar <a...|
|            NULL|                NULL|            NULL|          78|               72|<p>Usa <a href="h...|
|            NULL|                NULL|            NULL|         108|              149|<p>Puedes aplicar...|
|            NULL|                NULL|            NULL|         126|              326|<p>No es necesar

In [46]:
# Left Semi join
# El resultado incluyen los valores del primer DataFrame que existen en el segundo
joinType = "left_semi"
dfLSemi = dfRespuestas.join(dfPreguntas, joinExpression, joinType)
nFilas = dfLSemi.count()
print("Número de filas = {0}".format(nFilas))
dfLSemi.show(100)

Número de filas = 82491


+------------+-----------------+--------------------+
|ID Respuesta|Usuario respuesta|           Respuesta|
+------------+-----------------+--------------------+
|      380922|           175611|<pre><code>let us...|
|      381484|           105299|<p>El problema ra...|
|      381549|           187973|<p>Parece como si...|
|      381646|           163003|<p><strong>Prueba...|
|      382301|           184974|<p>Lo que podes h...|
|      382414|            44710|<p>El problema co...|
|      384638|           152272|<p>A priori sin u...|
|      384647|           103016|<p>Para el manejo...|
|      384994|           189662|<p>revisa lo sigu...|
|      385152|           133891|<p>Lo que está pa...|
|      385528|           147278|<p>Lo solucione d...|
|      386998|            62517|<p>Bien, he conse...|
|      390030|            43816|<p>Si lo que te m...|
|      391318|            55132|<p>Las variables ...|
|      392298|            54039|<p>No ejecuta la ...|
|      392791|           165

In [47]:
# Left Anti join
# El resultado incluyen los valores del primer DataFrame que NO existen en el segundo
joinType = "left_anti"
dfLAnti = dfRespuestas.join(dfPreguntas, joinExpression, joinType)
nFilas = dfLAnti.count()
print("Número de filas = {0}".format(nFilas))
dfLAnti.show(100)

Número de filas = 129124


+------------+-----------------+--------------------+
|ID Respuesta|Usuario respuesta|           Respuesta|
+------------+-----------------+--------------------+
|      379773|           156639|<p>Si te vas a <a...|
|      379892|           156982|<p>Creo que es po...|
|      380065|           187136|<p>El mejor conse...|
|      381626|            20709|<p><a href="https...|
|      382223|           183657|<p>Aquí puedes ve...|
|      382614|           188417|<p>Si quieres un ...|
|      383675|           188830|<p>El error se de...|
|      384113|           186392|<p>En lo personal...|
|      385424|           190276|<p>Existen dos mó...|
|      386599|           150924|<p>Lo que debería...|
|      387467|            29123|<p>Ok,</p><br/><p...|
|      388396|            73513|<p>Algo como esto...|
|      388859|            54039|<p>Aplica el esti...|
|      389131|           184983|<p>Te dejo este e...|
|      389639|            61388|<p>puedes hacerlo...|
|      390277|            81

In [48]:
# Cross join
# Producto cartesiano, une cada fila del primer DataFrame con todas las del segundo
# NO DEBE USARSE, EXTREMADAMENTE COSTOSO
dfCross = dfRespuestas.crossJoin(dfPreguntas)
nFilas = dfCross.count()
print("Número de filas = {0}".format(nFilas))
dfCross.show(100)

Número de filas = 17456332965
+------------+-----------------+--------------------+----------------+--------------------+----------------+
|ID Respuesta|Usuario respuesta|           Respuesta|Usuario pregunta|            Pregunta|ID Resp Aceptada|
+------------+-----------------+--------------------+----------------+--------------------+----------------+
|      379394|           186734|<p>Si quieres pru...|          146078|<p>soy principian...|          379397|
|      379394|           186734|<p>Si quieres pru...|          183610|<p>¿Alguien sabe ...|          379500|
|      379394|           186734|<p>Si quieres pru...|          152335|<p>estoy desarrol...|          379443|
|      379394|           186734|<p>Si quieres pru...|          117785|<p>Estudiando el ...|          379436|
|      379394|           186734|<p>Si quieres pru...|          138234|<p>tengo una cons...|          379442|
|      379394|           186734|<p>Si quieres pru...|          171228|<p>Necesito sumar...|       

## Funciones escalares y agregados

Spark ofrece un ámplio abanico de funciones para operar con los DataFrames:
- Funciones matemáticas: ``abs``, ``log``, ``hypot``, etc.
- Operaciones con strings: ``lenght``, ``concat``, etc.
- Operaciones con fechas: ``year``, ``date_add``, etc.
- Operaciones de agregación: ``min``, ``max``, ``count``, ``avg``, ``sum``, ``sumDistinct``, ``stddev``, ``variance``, ``kurtosis``, ``skewness``, ``first``, ``last``, ``window``, etc.

Una descripción de estas funciones se puede encontrar en <https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html>.

In [49]:
from pyspark.sql.functions import datediff, col
colUltimaActividad = col("LastActivityDate")
fechaCreacionCol = col("Fecha_de_creación")
# Buscamos la pregunta con respuesta aceptada que estuvo más tiempo activa
# (con la mayor diferencia entre los valores de LastActivityDate y Fecha de creacion)
masActiva = questionsWithAcceptedAnswersDf\
            .withColumn("tiempoActiva",
                        datediff(colUltimaActividad,
                                 fechaCreacionCol))\
            .orderBy("tiempoActiva", ascending=False)\
            .head()
print("La pregunta \n\n{0}\n\nestuvo activa {1} días".\
      format(masActiva.Body.replace("&lt;", "<").replace("&gt;", ">"), masActiva.tiempoActiva))

La pregunta 

<p>¿Cuál es la diferencia entre un <code>inner</code> y un <code>outer join</code>?</p><br/><p>Y ¿cuál es la función de los modificadores?</p><br/><ul><br/><li><code>left</code></li><br/><li><code>right</code></li><br/><li><code>full</code></li><br/></ul><br/>

estuvo activa 2809 días


In [50]:
from pyspark.sql.functions import window
# Obtenemos el número de post por semana de cada usuario
# Agrupamos por OwnerUserId y una ventana de fechas de creación de 1 semana
questionsWithAcceptedAnswersDf.groupBy(
                   colUserId, window(fechaCreacionCol, "1 week").alias("Semana"))\
                  .count()\
                  .sort("count", ascending=False)\
                  .show(20,False)

+-----------+------------------------------------------+-----+
|OwnerUserId|Semana                                    |count|
+-----------+------------------------------------------+-----+
|22584      |{2018-01-04 01:00:00, 2018-01-11 01:00:00}|18   |
|NULL       |{2017-12-28 01:00:00, 2018-01-04 01:00:00}|17   |
|NULL       |{2017-03-16 01:00:00, 2017-03-23 01:00:00}|16   |
|38304      |{2017-09-21 02:00:00, 2017-09-28 02:00:00}|15   |
|35888      |{2017-07-20 02:00:00, 2017-07-27 02:00:00}|14   |
|NULL       |{2016-09-01 02:00:00, 2016-09-08 02:00:00}|13   |
|NULL       |{2017-05-25 02:00:00, 2017-06-01 02:00:00}|13   |
|22584      |{2017-12-28 01:00:00, 2018-01-04 01:00:00}|13   |
|NULL       |{2017-11-02 01:00:00, 2017-11-09 01:00:00}|13   |
|22620      |{2016-11-24 01:00:00, 2016-12-01 01:00:00}|12   |
|24010      |{2017-04-13 02:00:00, 2017-04-20 02:00:00}|12   |
|22620      |{2016-12-01 01:00:00, 2016-12-08 01:00:00}|12   |
|22584      |{2017-05-11 02:00:00, 2017-05-18 02:00:00}

In [51]:
import pyspark.sql.functions as F

# Buscar la media y máximo de la columna "Score" de todas las filas y el número total del DataFrame completo.
dfSE.select(F.avg(colPuntos), F.max(colPuntos), F.count(colPuntos)).show()

+------------------+----------+------------+
|        avg(Score)|max(Score)|count(Score)|
+------------------+----------+------------+
|0.9099808454328786|       232|      410346|
+------------------+----------+------------+



In [52]:
# Otra forma usando describe
dfSE.select(colPuntos).describe().show()

+-------+------------------+
|summary|             Score|
+-------+------------------+
|  count|            410346|
|   mean|0.9099808454328786|
| stddev| 2.193521393477432|
|    min|               -28|
|    max|               232|
+-------+------------------+



## Tipos complejos

Spark permite trabajar con tres tipos de datos complejos: `structs`, `arrays` y `maps`

### Structs

DataFrames dentro de DataFrames

In [53]:
from pyspark.sql.functions import struct,col
# Creamos un nuevo DF con una columna que combina dos columnas existentes
colId = col("id")
colNumVistas = col("ViewCount")
colNRespuestas = col("AnswerCount")
dfStruct = dfSE.select(colId, colNumVistas, colNRespuestas,
                struct(colNumVistas, colNRespuestas)\
               .alias("Vistas_Respuestas"))
dfStruct.show(5)

+------+---------+-----------+-----------------+
|    id|ViewCount|AnswerCount|Vistas_Respuestas|
+------+---------+-----------+-----------------+
|379389|      592|          1|         {592, 1}|
|379391|     1434|          1|        {1434, 1}|
|379392|       18|          0|          {18, 0}|
|379393|      500|          0|         {500, 0}|
|379394|        0|          0|           {0, 0}|
+------+---------+-----------+-----------------+
only showing top 5 rows



In [54]:
dfStruct.printSchema()

root
 |-- id: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Vistas_Respuestas: struct (nullable = false)
 |    |-- ViewCount: integer (nullable = true)
 |    |-- AnswerCount: integer (nullable = true)



In [55]:
# Obtenemos un campo de la columna compuesta
dfStruct.select(col("Vistas_Respuestas").getField("ViewCount"))\
        .show(5)

+---------------------------+
|Vistas_Respuestas.ViewCount|
+---------------------------+
|                        592|
|                       1434|
|                         18|
|                        500|
|                          0|
+---------------------------+
only showing top 5 rows



### Arrays

Permiten trabajar con datos como si fuera un array Python

*Ejemplo*

Obtener el número de *tags* para cada pregunta con respuesta aceptada y eliminar los símbolos ``&lt;`` y ``&gt;``

  - Las "tags" de cada pregunta se guardan concatenadas, separadas por < y >, codificados como ``&lt;`` y ``&gt;``

`&lt;english-comparison&gt;&lt;translation&gt;&lt;phrase-request&gt;`

In [56]:
# Obtenemos un DataFrame sin tags nulas
dfSE.show(10)
dfNoNullTags = dfSE.dropna("any", subset=["Tags"])
dfNoNullTags.select("Tags").show(10, False)

+------+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+--------------------+---------+
|    Id|AcceptedAnswerId|AnswerCount|                Body|ClosedDate|CommentCount|CommunityOwnedDate|ContentLicense|   Fecha_de_creación|FavoriteCount|    LastActivityDate|        LastEditDate|LastEditorDisplayName|LastEditorUserId|OwnerDisplayName|OwnerUserId|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+------+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+------------

In [57]:
# Añado una columna con las etiquetas separadas
from pyspark.sql.functions import split
colTags = col("Tags")
dfTags = dfNoNullTags.withColumn("tag_array", split(colTags, "&gt;&lt;"))
dfTags.select(col("tag_array")).show(10, False)

+-------------------------------------------------+
|tag_array                                        |
+-------------------------------------------------+
|[<array><ionic>]                                 |
|[<javascript><html5><json><xml><api>]            |
|[<mysql>]                                        |
|[<php><mysql><android-studio><push-notification>]|
|[<c>]                                            |
|[<android-studio><kotlin><cache><android-glide>] |
|[<java><spring-boot><java-8><cookies>]           |
|[<php><html><jquery>]                            |
|[<java><array>]                                  |
|[<base-de-datos><modelorelacional>]              |
+-------------------------------------------------+
only showing top 10 rows



In [58]:
dfTags.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- Fecha_de_creación: timestamp (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- tag_array:

In [59]:
from pyspark.sql.functions import size
# Mostramos el número de etiquetas de cada entrada
colTag_array = col("tag_array")
dfTags.select(colTag_array, size(colTag_array)).show(5, False)

+-------------------------------------------------+---------------+
|tag_array                                        |size(tag_array)|
+-------------------------------------------------+---------------+
|[<array><ionic>]                                 |1              |
|[<javascript><html5><json><xml><api>]            |1              |
|[<mysql>]                                        |1              |
|[<php><mysql><android-studio><push-notification>]|1              |
|[<c>]                                            |1              |
+-------------------------------------------------+---------------+
only showing top 5 rows



In [60]:
# Mostramos la segunda etiqueta de cada entrada
dfTags.selectExpr("tag_array", "tag_array[1]").show(5, False)

+-------------------------------------------------+------------+
|tag_array                                        |tag_array[1]|
+-------------------------------------------------+------------+
|[<array><ionic>]                                 |NULL        |
|[<javascript><html5><json><xml><api>]            |NULL        |
|[<mysql>]                                        |NULL        |
|[<php><mysql><android-studio><push-notification>]|NULL        |
|[<c>]                                            |NULL        |
+-------------------------------------------------+------------+
only showing top 5 rows



In [61]:
from pyspark.sql.functions import array_contains
# Miramos si en las tags aparece la palabra "usage"
dfTags.withColumn("Con_usage", array_contains(colTag_array, "&lt;usage"))\
      .select(colTag_array, col("Con_usage")).show(5, False)

+-------------------------------------------------+---------+
|tag_array                                        |Con_usage|
+-------------------------------------------------+---------+
|[<array><ionic>]                                 |false    |
|[<javascript><html5><json><xml><api>]            |false    |
|[<mysql>]                                        |false    |
|[<php><mysql><android-studio><push-notification>]|false    |
|[<c>]                                            |false    |
+-------------------------------------------------+---------+
only showing top 5 rows



In [62]:
from pyspark.sql.functions import explode
# Convertimos cada etiqueta en una fila
dfTagsRows = dfTags.withColumn("Tags2", explode(colTag_array))
dfTagsRows.select(colTags, col("Tags2")).show(10, False)

+-----------------------------------------------+-----------------------------------------------+
|Tags                                           |Tags2                                          |
+-----------------------------------------------+-----------------------------------------------+
|<array><ionic>                                 |<array><ionic>                                 |
|<javascript><html5><json><xml><api>            |<javascript><html5><json><xml><api>            |
|<mysql>                                        |<mysql>                                        |
|<php><mysql><android-studio><push-notification>|<php><mysql><android-studio><push-notification>|
|<c>                                            |<c>                                            |
|<android-studio><kotlin><cache><android-glide> |<android-studio><kotlin><cache><android-glide> |
|<java><spring-boot><java-8><cookies>           |<java><spring-boot><java-8><cookies>           |
|<php><html><jquery>

In [63]:
# Elimina los símbolos &lt; y &gt;
from pyspark.sql.functions import regexp_replace
dfTags = dfTagsRows.withColumn("Tags_separadas",
            regexp_replace("Tags2", "&[l,g]t;", "")).drop("Tags2")
dfTags.select(colTags, col("Tags_separadas")).show(10, False)

+-----------------------------------------------+-----------------------------------------------+
|Tags                                           |Tags_separadas                                 |
+-----------------------------------------------+-----------------------------------------------+
|<array><ionic>                                 |<array><ionic>                                 |
|<javascript><html5><json><xml><api>            |<javascript><html5><json><xml><api>            |
|<mysql>                                        |<mysql>                                        |
|<php><mysql><android-studio><push-notification>|<php><mysql><android-studio><push-notification>|
|<c>                                            |<c>                                            |
|<android-studio><kotlin><cache><android-glide> |<android-studio><kotlin><cache><android-glide> |
|<java><spring-boot><java-8><cookies>           |<java><spring-boot><java-8><cookies>           |
|<php><html><jquery>

In [64]:
# Número de entradas con la etiqueta word-choice
print("Número de entradas con la etiqueta word-choice = {0}"
      .format(dfTags
      .filter(col("Tags_separadas") == "word-choice")
      .count()))

Número de entradas con la etiqueta word-choice = 0


## Funciones de ventana

Similares a las de funciones de agregación, permiten operar en grupos de filas devolviendo un único valor para cada fila. Esto permite, entre otras cosas:

  - Obtener medias móviles
  - Calcular sumas acumuladas
  - Acceder a los valores de una fila por encima de la actual

Básicamente, una función de ventana (window function) calcula un valor para cada fila de entrada de una tabla en base a un grupo de filas, denominado *frame*.

Como funciones de ventana se puede usar las funciones de agregación ya comentadas y otras funciones adicionales (``cume_dist``, ``dense_rank``, ``lag``, ``lead``, ``ntile``, ``percent_rank``, ``rank``, ``row_number``) especificadas como *Window functions* en <https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/window.html>

#### Ejemplo 1
A partir del DataFrame ``dfPregConRespAcept``, mostrar la puntuación (columna "Score") máxima por usuario, y, para cada pregunta, la diferencia de su puntuación con el máximo del usuario.


In [65]:
from pyspark.sql.window import Window

# Especificamos la ventana que particiona las filas por la columna OwnerUserId
ventana = Window.partitionBy(colUserId)
print(type(ventana))


<class 'pyspark.sql.window.WindowSpec'>


In [66]:
# Creamos una columna con los máximos valores de Score por usuario
colMaxPuntos = F.max(colPuntos).over(ventana)
print(type(colMaxPuntos))

<class 'pyspark.sql.column.Column'>


In [67]:
# Obtenemos un nuevo DataFrame incluyendo la puntuación máxima por usuario
# y la diferencia entre este máximo y la puntuación de cada pregunta
questionsWithAcceptedAnswersDf.select(colUserId, colId.alias("Pregunta"),
                          colPuntos, colMaxPuntos.alias("maxPorUsuario"))\
                  .withColumn("Diferencia", colMaxPuntos-colPuntos)\
                  .orderBy(colUserId, colId)\
                  .show(30)

+-----------+--------+-----+-------------+----------+
|OwnerUserId|Pregunta|Score|maxPorUsuario|Diferencia|
+-----------+--------+-----+-------------+----------+
|       NULL|    1455|    5|           40|        35|
|       NULL|    1493|   18|           40|        22|
|       NULL|    1630|    5|           40|        35|
|       NULL|    2046|    5|           40|        35|
|       NULL|    2431|    3|           40|        37|
|       NULL|    2441|    4|           40|        36|
|       NULL|    2956|    3|           40|        37|
|       NULL|    4713|    3|           40|        37|
|       NULL|    5211|    2|           40|        38|
|       NULL|    5790|    3|           40|        37|
|       NULL|    6011|    0|           40|        40|
|       NULL|    6041|    1|           40|        39|
|       NULL|    6508|    5|           40|        35|
|       NULL|    6872|    2|           40|        38|
|       NULL|    7744|    0|           40|        40|
|       NULL|    8135|    1|

#### Ejemplo 2
Mostrar para cada usuario y pregunta del DataFrame ``dfPregConRespAcept`` el número de días que pasaron desde la anterior pregunta del usuario hasta la actual, y desde esta hasta la siguiente.

In [68]:
# Especificamos la ventana que particiona las filas por la columna OwnerUserId y las ordena por fecha de creación
from pyspark.sql.window import WindowSpec

ventana: WindowSpec = Window.partitionBy(colUserId).orderBy(fechaCreacionCol)

In [69]:
# Creamos una columna que referencia a la pregunta anterior por fecha
colAnterior: Column = F.lag(fechaCreacionCol, 1).over(ventana)
# Creamos una columna que referencia a la pregunta posterior por fecha
colPosterior: Column = F.lead(fechaCreacionCol, 1).over(ventana)

# Mostramos para cada usuario y pregunta el id de la pregunta anterior y posterior
questionsWithAcceptedAnswersDf.select(colUserId, colId, fechaCreacionCol.alias("Fecha de creación"),
                          F.datediff(fechaCreacionCol,colAnterior).alias("Días desde"),
                          F.datediff(colPosterior,fechaCreacionCol).alias("Días hasta"))\
                  .orderBy(colUserId, colId)\
                  .show(30, truncate=False)

+-----------+-----+-----------------------+----------+----------+
|OwnerUserId|id   |Fecha de creación      |Días desde|Días hasta|
+-----------+-----+-----------------------+----------+----------+
|NULL       |1455 |2015-12-21 15:36:13.263|NULL      |1         |
|NULL       |1493 |2015-12-22 00:01:49.88 |1         |1         |
|NULL       |1630 |2015-12-23 03:54:26.557|1         |12        |
|NULL       |2046 |2016-01-04 18:09:54.537|12        |8         |
|NULL       |2431 |2016-01-12 17:03:14.863|8         |0         |
|NULL       |2441 |2016-01-12 19:35:58.44 |0         |9         |
|NULL       |2956 |2016-01-21 22:46:57.103|9         |34        |
|NULL       |4713 |2016-02-24 20:12:37.197|34        |9         |
|NULL       |5211 |2016-03-04 17:41:42.637|9         |11        |
|NULL       |5790 |2016-03-15 17:37:29.7  |11        |3         |
|NULL       |6011 |2016-03-18 18:50:14.727|3         |1         |
|NULL       |6041 |2016-03-19 12:13:42.133|1         |10        |
|NULL     

## Funciones definidas por el usuario (UDFs)

Si queremos una función que no está implementada, podemos crear nuestra propia función que opere sobre columnas.

  - Las UDFs en Python pueden ser bastante ineficientes, debido a la serialización de datos a Python
  - Preferible programarlas en Scala o Java (se pueden usar desde Python)


#### Ejemplo

Usar UDFs para obtener el número de *tags* para cada pregunta y cambiar los ``&lt;`` y ``&gt;`` por < y >

  - Las "tags" de cada pregunta se guardan concatenadas, separadas por < y >, codificados como ``&lt;`` y ``&gt;``

`&lt;english-comparison&gt;&lt;translation&gt;&lt;phrase-request&gt;`

Para contar el número de tags, basta con contar el número de apariciones de ``&lt;`` en el string.

In [70]:
colTags: Column = col("Tags")
# Obtenemos un DataFrame sin tags nulas
dfNoNullTags: DataFrame = dfSE.dropna("any", subset=["Tags"])

In [71]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Se puede hacer de dos formas, o bien con una función o
# con una anotación @udf

# Definimos una función que devuelva el número de &lt; en un string
@udf(returnType=IntegerType())
def udfCuentaTags(tags):
    return tags.count('&lt;')

# Definimos una función que reemplace &lt y &gt por < y >
def reemplazaTags(tags):
    return tags.replace('&lt;', '<').replace('&gt;', '>')

# Creamos udfs a partir de esta última función
udfReemplazaTags = udf(reemplazaTags)

In [72]:
dfNoNullTags.select(udfReemplazaTags(colTags).alias("Etiquetas"),\
                    udfCuentaTags(colTags).alias("nEtiquetas"))\
                  .show(truncate=False)


+-----------------------------------------------+----------+
|Etiquetas                                      |nEtiquetas|
+-----------------------------------------------+----------+
|<array><ionic>                                 |0         |
|<javascript><html5><json><xml><api>            |0         |
|<mysql>                                        |0         |
|<php><mysql><android-studio><push-notification>|0         |
|<c>                                            |0         |
|<android-studio><kotlin><cache><android-glide> |0         |
|<java><spring-boot><java-8><cookies>           |0         |
|<php><html><jquery>                            |0         |
|<java><array>                                  |0         |
|<base-de-datos><modelorelacional>              |0         |
|<php><javascript><jquery><laravel><eloquent>   |0         |
|<oracle><oracle-sqldeveloper>                  |0         |
|<angular>                                      |0         |
|<javascript><html5>    

In [73]:
# Llamo a las UDFs Scala usando una expresión (si estuvieran definidas en Scala)
#dfNoNullTags.selectExpr("udfReemplazaTagsSc(Tags) AS Etiquetas",
#                              "udfCuentaTagsSc(Tags) AS nEtiquetas")\
#                  .show(truncate=False)

## Uso de comandos SQL

Los comandos SQL ejecutados desde Spark se trasladan a operaciones sobre DataFrames

 - Se pueden ejecutar comandos remotos a través del servidor JDBC/ODBC [Thrift](https://spark.apache.org/docs/latest/sql-programming-guide.html#distributed-sql-engine)
 - También puede trabajar con datos almacenados en [Apache Hive](https://spark.apache.org/docs/latest/sql-programming-guide.html#hive-tables)

Para usar comandos SQL sobre un DataFrame, este tiene que registrarse como una *tabla* o *vista*

 - la vista puede crearse como temporal (desaparece al terminar la sesión) o global (se mantiene entre sesiones)


In [74]:
# Registra el DataFrame dfPregConRespAcept como una vista temporal
questionsWithAcceptedAnswersDf\
    .createOrReplaceTempView("tabla_PregConRespAcept")

# Crea una tabla con los datops guardados en Parquet
spark.sql("""CREATE TABLE tabla_SE
             USING PARQUET OPTIONS (path '/tmp/tcdm-public/datos/dfSE.parquet')""")

DataFrame[]

In [75]:
spark.sql("SELECT * FROM tabla_SE").printSchema()


root
 |-- Id: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- CommentCount: integer (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- Fecha_de_creación: timestamp (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: integer (nullable = true)



In [76]:
# Ejecuta un comando SQL sobre la tabla
dfUser100: DataFrame = spark.sql("""SELECT OwnerUserId,Id FROM tabla_SE
                         WHERE OwnerUserId >= 100""")
dfUser100.show(5)

+-----------+------+
|OwnerUserId|    Id|
+-----------+------+
|     146078|379389|
|     186742|379391|
|     131115|379392|
|     186748|379393|
|     186734|379394|
+-----------+------+
only showing top 5 rows



In [77]:
# Podemos ver las tablas creadas
spark.sql("SHOW TABLES").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|            tabla_se|      false|
|         |tabla_pregconresp...|       true|
+---------+--------------------+-----------+



In [78]:
# Podemos crear un nuevo DataFrame a partir de una de la tablas
dfFromTable = spark.sql("SELECT * FROM tabla_PregConRespAcept")
dfFromTable.show(5)

+------+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+-------------------+--------------------+---------+
|    Id|AcceptedAnswerId|AnswerCount|                Body|ClosedDate|CommentCount|CommunityOwnedDate|ContentLicense|   Fecha_de_creación|FavoriteCount|    LastActivityDate|        LastEditDate|LastEditorDisplayName|LastEditorUserId|OwnerDisplayName|OwnerUserId|ParentId|PostTypeId|Score|               Tags|               Title|ViewCount|
+------+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+-------------------+---------------

In [79]:
spark.sql("DROP TABLE IF EXISTS tabla_PregConRespAcept")
spark.sql("DROP TABLE IF EXISTS tabla_SE")

spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+

